# Imports


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

# Load the Fashion MNIST dataset

contains 70,000 grayscale images in 10 categories

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Loading the dataset returns four NumPy arrays:
  - train_images, train_labels are the training set
  - test_images, test_labels are the test set
  
images have a 28 x 28 size with pixel values from 0 to 255.
Labels are an array of integers, ranging from 0 to 9.
Each image is mapped to a single name, we have to define the class names:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Briefly Explore the data

In [ ]:
# training set shape:
print('Training set image dimension:',str(train_images.shape))
print('Training set label dimension:',str(train_labels.shape))

# test set shape:
print('Test set image dimension:',str(test_images.shape))
print('Test set label dimension:',str(test_labels.shape))

# Data preprocessing

In [ ]:
# Inspect one image
plt.figure()
plt.imshow(train_images[5000])
plt.colorbar()
plt.grid(False)
plt.show()


before feeding the data to the neural network model, we want to normalize them in a range of 0 to 1.

In [ ]:
train_images = train_images / train_images.max()

test_images = test_images / train_images.max()


In [ ]:
# Let's show new range
plt.figure()
plt.imshow(train_images[5000])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Let's show the first 25 images
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

#PART 1: Fully Connected Model

> Blocco con rientro






# Build the model: FILL THE CODE

Create a sequential model sequential consisting of

- Dense layer with 128 neurons and a relu activation
- Dense layer with 10 neurons and softmax activation

N.B. the input image must be flattened before being fed to the dense layers!


In [ ]:
# Keras sequential model
model = #...

In [ ]:
# Let's show the architecture of the model
model.summary()

# Compile the model: FILL THE CODE
during this step we need to select a few settings:
    
    - Loss function: measures how the model is accurate
      during training (is what we want to minimize)
    - Optimizer: how the model is updated based on the data
      that it sees and loss function
    - Metrics: Used to monitor training and testing steps:
      Accuracy fraction of correctly classified images
      
Use Adam optimizer and a sparse_categorical_crossentropy loss, , use accuracy as a metric
      


In [ ]:
#....

# Train the model: FILL THE CODE


To train the neural network model, we need to follow these steps:

    - 1. Feed training data to the model
    
    - 2. Model hopefully learns to associate images and labels

    - 3. Test model predictions on an "unseen" test set and verify accuracy
    
train the data for 10 epochs using the built-in training loop

In [ ]:
#...

# Evaluate Accuracy

In [ ]:
# Check how model performs on test dataset
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

# Make Predictions

In [ ]:
# Predictions over test set
predictions = model.predict(test_images)

In [ ]:
# Show result
img_idx = 0 # Idx of image
print('Model output:',predictions[img_idx])
print('Predicted label:', np.argmax(predictions[img_idx]))
print('Ground truth label:',test_labels[img_idx])

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

# Feature extraction
Let's check how we can extract the output of intermediate layers

In [ ]:
extractor = keras.Model(inputs=model.inputs,
                        outputs = [layer.output for layer in model.layers])
features = extractor(test_images)

In [ ]:
n_layers = len(model.layers)
layer_names = ['flatten','fc1','output']
num_images = 5
for i in range(0,num_images):
  plt.figure(figsize=(20, 2))
  plt.subplot(1,n_layers+1,1)
  plt.imshow(test_images[i], cmap=plt.cm.binary)
  for n_l in range(0,n_layers):
    plt.subplot(1,n_layers+1,n_l+2)
    #print(features[n_l].numpy().shape)
    plt.plot(features[n_l].numpy()[i])
    plt.title(layer_names[n_l])

#PART 2: Convolutional Model


#Image Resizing

Images are defined as 3D arrays consisting of $M$ rows, $N$ columns and $C$
channels

Two main type of images:


*   Grayscale images: $C=1$, the channel in this case may sometimes be omitted
*   RGB images: $C=3$, each channel corresponds to one color Red, Green or Blue

If we consider $I$ as the number of images, there are two ways to represent the images:

* Channel last: $I\times M \times N \times C$
* Channel first: $I\times \times C M \times N $

We will use channel last since it is the way that tensorflow prefers.


In [ ]:
print('Train images size:' +str(train_images.shape))
print('Test images size:' +str(test_images.shape))

In [ ]:
train_images = tf.expand_dims(train_images,axis=3)
test_images = tf.expand_dims(test_images,axis=3)

print('Train images size:' +str(train_images.shape))
print('Test images size:' +str(test_images.shape))

# Build the convolutional Network: FILL THE CODE

Create a sequential model sequential consisting of

- Conv2D layer with 64 filters and a relu activation
- Conv2D layer with 32 filters and a relu activation
- Dense layer with 10 neurons and a softmax activation           

(Use kernels of size (4,4), stride of size (2,2) and 'same' for the padding)

N.B. the input image must be flattened before being fed to the dense layers!
N.B. on the first layer input dimension must be specified


In [ ]:
# Keras sequential model
conv_model = #....

In [ ]:
# Let's show the architecture of the model
conv_model.summary()

## Compile the model: FILL THE CODE

use adam optimizer and sparse_categorical_crossentropy loss, use accuracy as a metric



In [ ]:
#...

## Train the model: FILL THE CODE

Train the convolutional model for 10 epochs using the built-in training loop

In [ ]:
#...

In [ ]:
# Check how model performs on test dataset
test_loss, test_acc = conv_model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

# Make predictions

In [ ]:
# Predictions over test set
predictions = conv_model.predict(test_images)

In [ ]:
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images[:,:,:,0])
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

# Feature Extraction

In [ ]:
extractor = keras.Model(inputs=conv_model.inputs,
                        outputs = [layer.output for layer in conv_model.layers])
features = extractor(test_images)

In [ ]:
n_layers = len(conv_model.layers)
layer_names = ['conv1','conv2','flatten','output']
num_images = 5
for i in range(0,num_images):
  plt.figure(figsize=(20, 2))
  plt.subplot(1,n_layers+1,1)
  plt.imshow(test_images[i,:,:,0], cmap=plt.cm.binary)
  for n_l in range(0,n_layers):
    plt.subplot(1,n_layers+1,n_l+2)
    if len(features[n_l].numpy().shape) == 4:
      plt.imshow(np.mean(features[n_l].numpy()[i,:,:,:], axis=2), cmap=plt.cm.binary)
    else:
      plt.plot(features[n_l].numpy()[i])
    plt.title(layer_names[n_l])

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE